In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt
import os
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import json
from os import listdir
import glob
from scipy import spatial
#from sklearn.metrics.pairwise import cosine_similarity
#from genism.models import Word2Vec
#print(os.listdir("../input"))
#print(os.listdir(".."))
#import spacy
#from spacy import displacy
#from topia.termextract import extract
# Any results you write to the current directory are saved as output.

In [4]:
df2=pd.read_csv("../dice_com-job_us_sample.csv")
print(df2.head())
jobs=[]
for job_title  in df2.jobtitle:
    if(job_title.lower() not in jobs):
        jobs.append(job_title)
#print(jobs)
job_description=np.asarray(df2.loc[:,"jobdescription"])
print(len(job_description[0:5]))
#nlp=spacy.load('en')

def remove_whitespace_entities(doc):
    doc.ents=[x for x in doc.ents if not (x.text.isspace())]
    return doc
#nlp.add_pipe(remove_whitespace_entities,after='ner')
#article=job_description[2]
#doc=nlp(article)
#article=[x for x in nlp(article) if not x.is_stop and x.pos!='PUNCT']
#article=[x.lemma_ for x in article]

#Using Named Entity Recognition
# displacy.render(nlp(str(article)), jupyter=True, style='ent')
# nlp(str(article)).ents

#Comparing similarity of each word with a given set of words. If the similarity score is high, the word is related to technology. Hence, it should be considered.
df_languages=pd.read_excel('./data/job_profile/languages.xlsx')
df_frameworks=pd.read_excel("./data/job_profile/frameworks.xlsx",header=None,error_bad_lines=False,delim_whitespace=True)
experience_regex=['\d+ years','\d+ experience','']
#print(df_frameworks)
frame=[str(x).split(",")[0] for x in df_frameworks.iloc[:,0]]
print(len(df_frameworks.columns))
dictionary=list(df_languages.iloc[:,0])
dictionary.extend(frame)
print(dictionary)
dictionary=[x.lower() for x in dictionary]
extracted_jobs=dict()
for i in range(len(job_description[:10000])):
    job_id=df2.iloc[i,-1]
    job=df2.iloc[i,3]
    flag=0
    for word in job.split(" "):
        word=word.lower()
        if word in dictionary:
            flag=1
            if job_id not in extracted_jobs.keys():
                extracted_jobs[job_id]=[]
            if word not in extracted_jobs[job_id]:
                extracted_jobs[job_id].append(word)
    if(flag==0):
        print(job_id)
print(extracted_jobs)
print(len(extracted_jobs))        
            
#doc_given_text=nlp(u'computer science')
#sample_word=nlp(u'Java')
#words=[]
#for ele in doc:
#    if(doc_given_text.similarity(ele)>0.5):
#        words.append(ele.text)
#print(words)
#print(doc_given_text.similarity(sample_word))
#print(doc_given_text.similarity(doc[3]))
#print(doc[3].vector)


                                       advertiserurl  \
0  https://www.dice.com/jobs/detail/AUTOMATION-TE...   
1  https://www.dice.com/jobs/detail/Information-S...   
2  https://www.dice.com/jobs/detail/Business-Solu...   
3  https://www.dice.com/jobs/detail/Java-Develope...   
4  https://www.dice.com/jobs/detail/DevOps-Engine...   

                             company  \
0  Digital Intelligence Systems, LLC   
1  University of Chicago/IT Services   
2               Galaxy Systems, Inc.   
3                      TransTech LLC   
4                   Matrix Resources   

                            employmenttype_jobstatus  \
0  C2H Corp-To-Corp, C2H Independent, C2H W2, 3 M...   
1                                          Full Time   
2                                          Full Time   
3                                          Full Time   
4                                          Full Time   

                                      jobdescription               jobid  \
0  Lookin

In [ ]:
df2=pd.read_csv("../input/us-technology-jobs-on-dicecom/dice_com-job_us_sample.csv")
print(df2.head())
jobs=[]
for job_title  in df2.jobtitle:
    if(job_title.lower() not in jobs):
        jobs.append(job_title)
#print(jobs)
job_skills=np.asarray(df2.loc[:,"skills"])
print(len(job_description[0:5]))
#nlp=spacy.load('en')

def remove_whitespace_entities(doc):
    doc.ents=[x for x in doc.ents if not (x.text.isspace())]
    return doc
#nlp.add_pipe(remove_whitespace_entities,after='ner')
#article=job_description[2]
#doc=nlp(article)
#article=[x for x in nlp(article) if not x.is_stop and x.pos!='PUNCT']
#article=[x.lemma_ for x in article]

#Using Named Entity Recognition
# displacy.render(nlp(str(article)), jupyter=True, style='ent')
# nlp(str(article)).ents

#Tokenizing words
extracted_skills=dict()
training_range=int(0.7*len(job_skills))
for i in range(training_range):
    #print(i)
    #Method 1: Manual pre-processing
    job_id=df2.iloc[i,-1]
#     job=df2.iloc[i,-2]
#     words=job_skills[i].split(",")
#     if(words[0].lower()=="(see job description)"):
#         continue
#     #print(words)
#     #print(len(words))
#     for i in range(len(words)):
#         #print(type(word))
#         chunk=words[i].split("/")
#         words.remove(words[i])
#         words.extend(chunk)
            
#     extracted_skills[job_id]=[]
#     extracted_skills[job_id].extend(words)
    #Method 2:Using NLTK
    tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
    #print(job_skills[i])
    if(pd.isnull(job_skills[i])):
        continue
    stopwords_list=stopwords.words("english")
    tokens=re.split("|".join([","," and","/"," AND"," or"," OR",";"]),job_skills[i])
    tokens=list(set(tokens))
    #tokens=[x for x in tokens if x.isalpha()==True]
    #tokens=tokenizer.tokenize(job_skills[i])
    #print(tokens)
    #tokens=nltk.word_tokenize(job_skills[i])
    #print(stopwords)
    #stopwords_list=stopwords
    #words=[x for x in tokens if x not in stopwords_list]
    #print(tokens)
    extracted_skills[job_id]=[]
    extracted_skills[job_id].extend(tokens)
    #print(extracted_skills[job_id])
print(extracted_skills)    
    

In [23]:
def cosine_similarity(arr1,arr2):
    return 1 - spatial.distance.cosine(arr1, arr2)
class job_postings:    
    def __init__(self,link):
        self.df2=pd.read_csv(link)
    def clean_skills(self,training_range):
        extracted_skills=dict()
        job_skills=np.asarray(self.df2.loc[:,"skills"])
        for i in range(training_range):
            #print(i)
            #Method 1: Manual pre-processing
            job_id=self.df2.iloc[i,-1]
            #Method 2:Using NLTK
            tokenizer=nltk.tokenize.RegexpTokenizer(r'\w+')
            #print(job_skills[i])
            if(pd.isnull(job_skills[i])):
                continue
            stopwords_list=stopwords.words("english")
            tokens=re.split("|".join([","," and","/"," AND"," or"," OR",";"]),job_skills[i])
            tokens=list(set(tokens))
            extracted_skills[job_id]=[]
            extracted_skills[job_id].extend(tokens)
        return extracted_skills
    def extract_skills(self,extracted_skills):
        df_languages=pd.read_excel('./data/job_profile/languages.xlsx')
        df_frameworks=pd.read_csv("./data/job_profile/frameworks.csv")
        df_database=pd.read_csv("./data/job_profile/database.csv")
        df_os=pd.read_csv("./data/job_profile/operating_systems.csv")
        df_plat=pd.read_csv("./data/job_profile/platforms.csv")
        frameworks=df_frameworks.iloc[:,1].tolist()
        frameworks=[x.lower().strip() for x in frameworks]
        #frameworks=[str(x).split(",")[0] for x in df_frameworks.iloc[:,1]]
        languages=list(df_languages.iloc[:,0])
        languages=[x.lower().strip() for x in languages]
        #frameworks=[x.lower().strip().split('\t')[0] for x in frameworks]
        databases=df_database.iloc[:,0].tolist()
        databases=[x.lower().strip() for x in databases]
        op_systems=df_os.iloc[:,0].tolist()
        op_systems=[x.lower().strip() for x in op_systems]
        platforms=df_plat.iloc[:,1].tolist()
        #print(platforms)
        platforms=[x.lower().strip() for x in platforms]
        #print(frameworks)
        new_extracted=dict()
        for ele in extracted_skills.keys():
            final_lang=''
            final_frame=''
            final_others=''
            final_database=''
            final_plat=''
            final_os=''
            #print(extracted_skills[ele])
            for skill in extracted_skills[ele]:
                skill_base=skill.lower().strip()
                #print(skill_base)
                if(skill_base in languages):
                    if(final_lang==''):
                        final_lang=skill_base
                    else:
                        final_lang=final_lang+","+skill_base
                elif(skill_base in frameworks):
                    if(final_frame==''):
                        final_frame=skill_base
                    else:
                        final_frame=final_frame+","+skill_base
                elif(skill_base in databases):
                    if(final_database==''):
                        final_database=skill_base
                    else:
                        final_database=final_database+","+skill_base
                elif(skill_base in op_systems):
                    if(final_os==''):
                        final_os=skill_base
                    else:
                        final_os=final_os+","+skill_base
                elif(skill_base in platforms):
                    if(final_plat==''):
                        final_plat=skill_base
                    else:
                        final_plat=final_plat+","+skill_base
                else:
                    if(final_others==''):
                        final_others=skill_base
                    else:
                        final_others=final_others+","+skill_base
            new_extracted[ele]=[final_lang,final_frame,final_database,final_os,final_plat,final_others]
        extracted_skills_df=pd.DataFrame.from_dict(new_extracted,orient='index',columns=['Language','Framework','Database','OS','Platform','Others'])
        return extracted_skills_df
    def create_job_profile(self,extracted_skills_df,domain_df):
        job_id=extracted_skills_df.index.tolist()
        languages_df=pd.DataFrame(index=job_id)
        platforms_df=pd.DataFrame(index=job_id)
        frameworks_df=pd.DataFrame(index=job_id)
        databases_df=pd.DataFrame(index=job_id)
        
        for job,lang,frame,plat,datab in list(zip(job_id,extracted_skills_df.loc[:,'Language'].tolist(),extracted_skills_df.loc[:,'Framework'].tolist(),extracted_skills_df.loc[:,'Platform'].tolist(),extracted_skills_df.loc[:,'Database'].tolist())):
            #Languages
            l=lang.split(",")
            if(lang!=np.nan or lang!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in languages_df.columns):
                        #languages.append(ele)
                        languages_df[ele]=np.nan
                    languages_df.loc[job,ele]=1
            
            #Frameworks
            l=frame.split(",")
            if(frame!=np.nan or frame!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in frameworks_df.columns):
                        #languages.append(ele)
                        frameworks_df[ele]=np.nan
                    frameworks_df.loc[job,ele]=1

            #Platforms
            l=plat.split(",")
            if(plat!=np.nan or plat!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in platforms_df.columns):
                        #languages.append(ele)
                        platforms_df[ele]=np.nan
                    platforms_df.loc[job,ele]=1
            
            #Databases
            l=datab.split(",")
            if(datab!=np.nan or datab!=''):
                for ele in l:
                    if(ele==''):
                        continue
                    if(ele not in databases_df.columns):
                        #languages.append(ele)
                        databases_df[ele]=np.nan
                    databases_df.loc[job,ele]=1
        languages_df=languages_df.reindex_axis(sorted(languages_df.columns), axis=1)
        frameworks_df=frameworks_df.reindex_axis(sorted(frameworks_df.columns), axis=1)
        platforms_df=platforms_df.reindex_axis(sorted(platforms_df.columns), axis=1)
        databases_df=databases_df.reindex_axis(sorted(databases_df.columns), axis=1)
        domain_df=domain_df.reindex_axis(sorted(domain_df.columns), axis=1)

        languages_df.to_csv("./data/job_profile/languages_job_profile.csv")
        frameworks_df.to_csv("./data/job_profile/frameworks_job_profile.csv")
        platforms_df.to_csv("./data/job_profile/platforms_job_profile.csv")
        databases_df.to_csv("./data/job_profile/databases_job_profile.csv")
        domain_df.to_csv("./data/job_profile/domain_job_profile.csv")
        print(languages_df.columns)
        
    def clean_common_profile(self,df_user,df_job,flag):
        #Shift .net from languages to frameworks
        if(flag=='Language'):
            print(df_job.columns.tolist())
            #bash and bash/shell
            count=0
            for ele in df_user.loc[:,'bash/shell']:
                if(ele==1.0):
                    df_user.ix[count,'bash']=1.0
                count=count+1
            df_user.drop('bash/shell',axis=1)
            count=0
            for ele in df_job.loc[:,'bash/shell']:
                if(ele==1.0):
                    df_job.ix[count,'bash']=1.0
                count=count+1
            df_job.drop('bash/shell',axis=1)

        if(flag=='Framework'):
            print(df_user.columns.tolist())
            count=0
            for ele in df_user.loc[:,'nodejs']:
                if(ele==1.0):
                    df_user.ix[count,'node.js']=1.0
                count=count+1
            df_user.drop('nodejs',axis=1)
            count=0
            for ele in df_job.loc[:,'nodejs']:
                if(ele==1.0):
                    df_job.ix[count,'node.js']=1.0
                count=count+1
            df_job.drop('nodejs',axis=1)
            
        if(flag=='Platform'):
            print(df_user.columns.tolist())
        if(flag=='Database'):
            print(df_user.columns.tolist())
            count=0
            for ele in df_user.loc[:,'microsoft sql server']:
                if(ele==1.0):
                    df_user.ix[count,'sql server']=1.0
                count=count+1
            df_user.drop('microsoft sql server',axis=1)
            count=0
            for ele in df_job.loc[:,'microsoft sql server']:
                if(ele==1.0):
                    df_job.ix[count,'sql server']=1.0
                count=count+1
            df_job.drop('microsoft sql server',axis=1)

    #Input is two dataframes    
    def create_common_profile(self,job_profile_path,user_profile_path,output_path):
        #Domain
        userprofile=pd.read_csv(user_profile_path+"DevType.csv",index_col='Respondent')
        jobprofile=pd.read_csv(job_profile_path+"domain_job_profile.csv",index_col='Unnamed: 0')
        print("Read from file")
        print(jobprofile.index)
        #jobprofile=jobprofile.reset_index()
        #userprofile=userprofile.reset_index()
        userprofile.drop('Unnamed: 0', axis=1, inplace=True)
        jobprofile.drop('uniq_id', axis=1, inplace=True)
        jobprofile.index.name='uniq_id'
        print("index 2in domain")
        print(jobprofile.index)
        #print(jobprofile.loc[:,'uniq_id'])
        userprofile.rename(columns={'Product manager':'Product Manager','Back-end developer':'Back End','C-suite executive (CEO, CTO, etc.)':'C-suite executive','Data scientist or machine learning specialist':'Data Scientist','Database administrator':'Database Administrator(DBA)','Mobile developer':'Mobile Developer','Desktop or enterprise applications developer':'Enterprise application','DevOps specialist':'DevOps','Front-end developer':'Front End','Full-stack developer':'Full stack','Marketing or sales professional':'Sales professional','QA or test developer':'QA/Test Developer','System administrator':'System Administrator','Game or graphics developer':'Game developer'},inplace=True)
        jobprofile.rename(columns={'Business analyst':'Data or business analyst'},inplace=True)
        print(userprofile.columns)
        print(jobprofile.columns)
        print("index in domain")
        print(jobprofile.index)
        #Present in userprofile but not in jobprofile
        a=list(set(userprofile.columns)-set(jobprofile.columns))
        print(a)
        for i in a:
            if(i!='Respondent'):
                jobprofile[i]=0
        b=list(set(jobprofile.columns)-set(userprofile.columns))
        print(b)
        for i in b:
            if(i!='uniq_id'):
                userprofile[i]=0
        #userprofile=userprofile.set_index('Respondent')
        #jobprofile=jobprofile.set_index('uniq_id')
        userprofile=userprofile[sorted(userprofile.columns.tolist())]
        jobprofile=jobprofile[sorted(jobprofile.columns.tolist())]
        #Exclude 
        
        print(userprofile.columns==jobprofile.columns)
        
        print(userprofile.columns)
        print(jobprofile.columns)
        userprofile=userprofile[userprofile.columns.tolist()]
        jobprofile=jobprofile[jobprofile.columns.tolist()]
        userprofile.to_csv(output_path+"domain_user_profile.csv")
        jobprofile.to_csv(output_path+"domain_job_profile.csv")
        
        #Languages
        df_user=pd.read_csv(user_profile_path+"LanguageWorkedWith.csv",index_col='Respondent')
        df_job=pd.read_csv(job_profile_path+"languages_job_profile.csv",index_col=0)
        df_job.index.name='uniq_id'
        print("index is")
        print(df_job.index)
        print(df_user.columns)
        print(df_job.columns)
        df_user.drop('Unnamed: 0', axis=1, inplace=True)
        #df_job.drop('Unnamed: 0', axis=1, inplace=True)
        df_job.rename(columns={'visual basic .net':'vb.net'},inplace=True)
        df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
        df_job.columns=list(map(lambda x:x.lower(),df_job.columns))
        columns_to_add=[]
        a=list(set(df_user.columns)-(set(df_job.columns)))
        print(a)
        for i in a:
            if(i!='Respondent'):
                df_job[i]=0        
        b=list(set(df_job.columns)-set(df_user.columns))
        print(b)
        for i in b:
            if(i!='uniq_id'):
                df_user[i]=0
        print(df_job.index)        
        df_user=df_user[sorted(df_user.columns.tolist())]
        df_job=df_job[sorted(df_job.columns.tolist())]
        #df_user=userprofile.reindex_axis(sorted(df_user.columns), axis=1)
        #df_job=jobprofile.reindex_axis(sorted(df_job.columns), axis=1)
        print("index 2")
        print(df_job.index)
        print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
        self.clean_common_profile(df_user,df_job,'Language')
        print("language is")
        print(df_job.index[0])
        print(df_job.loc[df_job.index[0],:])
        df_user.to_csv(output_path+"languages_profile_user.csv")
        df_job.to_csv(output_path+"languages_profile_job.csv")
        
        #Frameworks
        df_user=pd.read_csv(user_profile_path+"FrameworkWorkedWith.csv",index_col='Respondent')
        df_job=pd.read_csv(job_profile_path+"frameworks_job_profile.csv",index_col=0) 
        df_job.index.name='uniq_id'
        print(df_user.columns)
        print(df_job.columns)
        df_user.drop('Unnamed: 0', axis=1, inplace=True)
        #df_job.drop('Unnamed: 0', axis=1, inplace=True)
        #df_job.rename(columns={'visual basic .net':'vb.net'},inplace=True)
        df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
        df_job.columns=list(map(lambda x:x.lower(),df_job.columns))
        
        a=list(set(df_user.columns)-(set(df_job.columns)))
        print(a)
        for i in a:
            if(i!='Respondent'):
                df_job[i]=0        
        b=list(set(df_job.columns)-set(df_user.columns))
        print(b)
        for i in b:
            if(i!='uniq_id'):
                df_user[i]=0
        #userprofile=userprofile.reindex_axis(sorted(userprofile.columns), axis=1)
        #jobprofile=jobprofile.reindex_axis(sorted(jobprofile.columns), axis=1)
        df_user=df_user[sorted(df_user.columns.tolist())]
        df_job=df_job[sorted(df_job.columns.tolist())]
    
        print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
        self.clean_common_profile(df_user,df_job,'Framework')   
        df_user.to_csv(output_path+"frameworks_profile_user.csv")
        df_job.to_csv(output_path+"frameworks_profile_job.csv")
        
        #Platforms
        df_user=pd.read_csv(user_profile_path+"PlatformWorkedWith.csv",index_col='Respondent')
        df_job=pd.read_csv(job_profile_path+"platforms_job_profile.csv",index_col=0) 
        print(df_user.columns)
        df_job.index.name='uniq_id'
        print(df_job.columns)
        df_user.drop('Unnamed: 0', axis=1, inplace=True)
        #df_job.drop('Unnamed: 0', axis=1, inplace=True)
        #df_job.rename(columns={'visual basic .net':'vb.net'},inplace=True)
        df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
        df_job.columns=list(map(lambda x:x.lower(),df_job.columns))
        
        a=list(set(df_user.columns)-(set(df_job.columns)))
        print(a)
        for i in a:
            if(i!='Respondent'):
                df_job[i]=0
        b=list(set(df_job.columns)-set(df_user.columns))
        print(b)
        for i in b:
            if(i!='uniq_id'):
                df_user[i]=0
        df_user=df_user[sorted(df_user.columns.tolist())]
        df_job=df_job[sorted(df_job.columns.tolist())]

        print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
        self.clean_common_profile(df_user,df_job,'Platform')        
        df_user.to_csv(output_path+"platforms_profile_user.csv")
        df_job.to_csv(output_path+"platforms_profile_job.csv")

        #Databases
        df_user=pd.read_csv(user_profile_path+"DatabaseWorkedWith.csv",index_col='Respondent')
        df_job=pd.read_csv(job_profile_path+"databases_job_profile.csv",index_col=0) 
        df_job.index.name='uniq_id'
        print(df_user.columns)
        print(df_job.columns)
        df_user.drop('Unnamed: 0', axis=1, inplace=True)
        #df_job.drop('Unnamed: 0', axis=1, inplace=True)
        #df_job.rename(columns={'visual basic .net':'vb.net'},inplace=True)
        df_user.columns=list(map(lambda x:x.lower(),df_user.columns))
        df_job.columns=list(map(lambda x:x.lower(),df_job.columns))
        
        a=list(set(df_user.columns)-(set(df_job.columns)))
        print(a)
        for i in a:
            if(i!='Respondent'):
                df_job[i]=0
        b=list(set(df_job.columns)-set(df_user.columns))
        print(b)
        for i in b:
            if(i!='uniq_id'):
                df_user[i]=0
        df_user=df_user[sorted(df_user.columns.tolist())]
        df_job=df_job[sorted(df_job.columns.tolist())]

        print(len(set(df_user.columns).intersection(df_job.columns)),len(df_user.columns))
        self.clean_common_profile(df_user,df_job,'Database')        
        df_user.to_csv(output_path+"databases_profile_user.csv")
        df_job.to_csv(output_path+"databases_profile_job.csv")
#flag indicates that a new user profile is to be passed as an argument. args represents the userdomain,languages,frameworks,platforms,databases respectively
#The length of each vector should be equal to the length of the job profile vector's languages, frameworks etc.
    def match_profile(self,input_path,user_id,flag=0,*args):
        #Match a given user_id with all jobs in the database
        
        #Check if user id exists
        df=pd.read_csv(input_path+"domain_user_profile.csv",index_col='Respondent')
        print(df.columns)
        matches=dict()
        if(flag==0):
            if(user_id in df.index):
                userdomain=df.loc[user_id,:]
                print(userdomain)
                #If it does, retrieve the user profile from input_path
                df=pd.read_csv(input_path+"languages_profile_user.csv",index_col='Respondent')
                userlanguages=df.loc[user_id,:]

                df=pd.read_csv(input_path+"frameworks_profile_user.csv",index_col='Respondent')
                userframeworks=df.loc[user_id,:]

                df=pd.read_csv(input_path+"platforms_profile_user.csv",index_col='Respondent')
                userplatforms=df.loc[user_id,:]

                df=pd.read_csv(input_path+"databases_profile_user.csv",index_col='Respondent')
                userdatabases=df.loc[user_id,:]

                userdomain=np.asarray(userdomain.fillna(0))
                userlanguages=np.asarray(userlanguages.fillna(0))
                userframeworks=np.asarray(userframeworks.fillna(0))
                userplatforms=np.asarray(userplatforms.fillna(0))
                userdatabases=np.asarray(userdatabases.fillna(0))
            #If it doesn't,take user profile as input

        #Args represents the user profile's domain, languages,frameworks,platforms,databases
        else:
            userdomain=np.asarray(arg[0].fillna(0))
            userlanguages=np.asarray(arg[1].fillna(0))
            userframeworks=np.asarray(arg[2].fillna(0))
            userplatforms=np.asarray(arg[3].fillna(0))
            userdatabases=np.asarray(arg[4].fillna(0))

        jobdomain=pd.read_csv(input_path+"domain_job_profile.csv",index_col='uniq_id')
        joblanguages=pd.read_csv(input_path+'languages_profile_job.csv',index_col='uniq_id')
        jobframeworks=pd.read_csv(input_path+'frameworks_profile_job.csv',index_col='uniq_id')
        jobplatforms=pd.read_csv(input_path+'platforms_profile_job.csv',index_col='uniq_id')
        jobdatabases=pd.read_csv(input_path+'databases_profile_job.csv',index_col='uniq_id')
        print(len(jobdomain.index),len(joblanguages.index))
        for i in jobdomain.index:
            print(i)
            domain=jobdomain.loc[i,:].fillna(0)
            language=joblanguages.loc[i,:].fillna(0)
            framework=jobframeworks.loc[i,:].fillna(0)
            platform=jobplatforms.loc[i,:].fillna(0)
            database=jobdatabases.loc[i,:].fillna(0)
            job_id=str(i)
            domain=np.asarray(domain)
            language=np.asarray(language)
            framework=np.asarray(framework)
            platform=np.asarray(platform)
            database=np.asarray(database)
            print(len(domain),len(userdomain))
            score=(0.7*cosine_similarity(domain,userdomain))+(0.3*(cosine_similarity(language,userlanguages)+cosine_similarity(framework,userframeworks)+cosine_similarity(platform,userplatforms)+cosine_similarity(database,userdatabases)))
            matches[job_id]=score
        matches=sorted(matches.items(),key=lambda x:x[1],reverse=True)
        recommendations=matches[:10]
        rows=pd.DataFrame(columns=self.df2.columns)
        count=0
        for i in recommendations:
            row=self.df2[self.df2['uniq_id']==i[0]]
            #rows[count]=np.asarray(row.values.T.tolist()[0])
            rows=rows.append(row.iloc[0])
            count=count+1
            print(row)
        return rows
            

obj=job_postings("../dice_com-job_us_sample.csv")
#training_range=int(0.7*len(job_skills))
#extracted_skills=obj.clean_skills(training_range)
#extracted_skills_df=obj.extract_skills(extracted_skills)
#print(extracted_skills_df)
#domain_df=pd.read_csv("../input/domain-df/preprocessed_df.csv")
#obj.create_job_profile(extracted_skills_df,domain_df)
#extracted_skills_df.to_csv("skill_extracted.csv")
obj.create_common_profile("../data_from_kaggle/job_profile/","./data/user_profile/","./data/")

#Path represents the location where final job and user profiles
rows=obj.match_profile("./data/",3)

Read from file
Index(['418ff92580b270ef4e7c14f0ddfc36b4', '8aec88cba08d53da65ab99cf20f6f9d9',
       '46baa1f69ac07779274bcd90b85d9a72', '3941b2f206ae0f900c4fba4ac0b18719',
       '45efa1f6bc65acc32bbbb953a1ed13b7', 'e0ac9d926dda5e95162ef05adea7318c',
       'e7e326053c586bd94e59f1fd74de4a1b', 'b0dadecf4c3c2beecb9c773ca11ecda4',
       '28f5e0c1cc3314813e674f0c32b04d1b', '95c9127e2770172f454f3b83981eaa88',
       ...
       '58920bf56320a30686832e5808bc10ba', 'a08db8b527bc82e3b464646732b837f1',
       'df62b009ce13775a21760bfdbdb095f1', '655351c499dcaf7f71efbf32bc2a76ee',
       'ee9f93d310f009dc9c0487ad2052b5ca', 'f9894a65fba83e7d248ba021253856b1',
       '9c5aa9bc9466d7269523f2b0f88d33f5', '120809874823e327aba99e9bd99e4861',
       'd3f81af99910f324ad114ccb4a82d71d', '28401827992cda8efc82d247cdc7fbfe'],
      dtype='object', length=15399)
index 2in domain
Index(['418ff92580b270ef4e7c14f0ddfc36b4', '8aec88cba08d53da65ab99cf20f6f9d9',
       '46baa1f69ac07779274bcd90b85d9a72', '3941b2f

C:\Users\Tejas\Anaconda3\lib\site-packages\ipykernel_launcher.py:161: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


language is
418ff92580b270ef4e7c14f0ddfc36b4
.net              NaN
abap              NaN
abc               NaN
actionscript      NaN
ada               NaN
ajax              NaN
apex              NaN
apl               NaN
arc               NaN
arduino           NaN
asp               NaN
assembly          NaN
atlas             NaN
automator         NaN
avenue            NaN
awk               NaN
bash              NaN
bash/shell        0.0
bc                NaN
bourne shell      1.0
bro               NaN
c                 NaN
c shell           NaN
c#                NaN
c++               NaN
caml              NaN
ceylon            NaN
cfml              NaN
ch                NaN
clarion           NaN
                 ... 
rust              NaN
s                 NaN
s-plus            NaN
sas               NaN
scala             NaN
sed               NaN
self              NaN
shell             NaN
signal            NaN
simula            NaN
simulink          NaN
smarty            NaN
spark    

C:\Users\Tejas\Anaconda3\lib\site-packages\ipykernel_launcher.py:182: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


Index(['Unnamed: 0', 'Salesforce', 'WordPress', 'Firebase', 'iOS', 'Android',
       'Predix', 'Mainframe', 'Amazon Echo', 'Azure', 'IBM Cloud or Watson',
       'Windows Desktop or Server', 'SharePoint', 'Apple Watch or Apple TV',
       'Google Cloud Platform/App Engine', 'Gaming console', 'AWS', 'Mac OS',
       'Raspberry Pi', 'Heroku', 'Google Home', 'Serverless', 'Windows Phone',
       'Arduino', 'Linux', 'Drupal', 'ESP8266'],
      dtype='object')
Index(['amazon echo', 'android', 'arduino', 'aws', 'azure', 'drupal',
       'firebase', 'gaming console', 'heroku', 'ios', 'linux', 'mac os',
       'mainframe', 'raspberry pi', 'salesforce', 'sharepoint',
       'windows phone', 'wordpress'],
      dtype='object')
['ibm cloud or watson', 'serverless', 'esp8266', 'google home', 'predix', 'apple watch or apple tv', 'google cloud platform/app engine', 'windows desktop or server']
[]
26 26
['amazon echo', 'android', 'apple watch or apple tv', 'arduino', 'aws', 'azure', 'drupal', 'esp826

C:\Users\Tejas\Anaconda3\lib\site-packages\ipykernel_launcher.py:199: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix


Index(['Back End', 'C-suite executive', 'Data Scientist',
       'Data or business analyst', 'Database Administrator(DBA)', 'Designer',
       'DevOps', 'Educator or academic researcher',
       'Embedded applications or devices developer', 'Engineering manager',
       'Enterprise application', 'Front End', 'Full stack', 'Game developer',
       'Information Security', 'Mobile Developer', 'Network Engineer',
       'Product Manager', 'QA/Test Developer', 'Sales professional',
       'Software Developer/Java Developer', 'Student', 'System Administrator',
       'Web Developer'],
      dtype='object')
Back End                                      NaN
C-suite executive                             NaN
Data Scientist                                NaN
Data or business analyst                      NaN
Database Administrator(DBA)                   1.0
Designer                                      NaN
DevOps                                        1.0
Educator or academic researcher           

C:\Users\Tejas\Anaconda3\lib\site-packages\scipy\spatial\distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


24 24
e7e326053c586bd94e59f1fd74de4a1b
24 24
b0dadecf4c3c2beecb9c773ca11ecda4
24 24
28f5e0c1cc3314813e674f0c32b04d1b
24 24
95c9127e2770172f454f3b83981eaa88
24 24
5e0ff38f5eaf44726f4e3d1dd257a244
24 24
e4a1ff1b6c0fda5f345e57cf1acb40dd
24 24
d0c81a2e3e5d666f3d730f1048c49132
24 24
51279d060da242e3baea98b26ddd641e
24 24
48b7ca0c2f6191fb48e7ecf19fbd3322
24 24
9e5704d08bc07ddb6df9ef98b223b036
24 24
e4dceaaaaae37ca6eabb04a7d42498b4
24 24
7ad2eadde69e07fee0e38c1a251dd81f
24 24
f3af6886ca0d133abda2ddf9b84633e4
24 24
e4f57bc5366124a0a47cac27f557f9ec
24 24
d3073d47b79938269b22bdea4dc0b9b8
24 24
c014c8569c91f22bc871940bfa4b4296
24 24
b7fab2d3de5e129310b382d8f51508f6
24 24
4868383e3f99535778f354fcb734d57c
24 24
81f8732626888727fcc1093b6c084839
24 24
cf902dc6fd702207563ed1cdc03ef05e
24 24
1bef3cf6259e80d18222407858ad9052
24 24
72a3f0f0f24a0a78c5fac3c2376f5979
24 24
4a3f8742fae151757eb290a774968371
24 24
b1db84e23f424b1481b29212749abadc
24 24
51a0050d469e787dba2102383797536d
24 24
f2ade5d053b992593e2

In [24]:
print(rows)
#Validation
#KMeans clustering



#Job-job similarity->Each user's recommendations->User-user similarity

                                       advertiserurl  \
0  https://www.dice.com/jobs/detail/AUTOMATION-TE...   
1  https://www.dice.com/jobs/detail/Information-S...   
2  https://www.dice.com/jobs/detail/Business-Solu...   
3  https://www.dice.com/jobs/detail/Java-Develope...   
4  https://www.dice.com/jobs/detail/DevOps-Engine...   
5  https://www.dice.com/jobs/detail/SAP-FICO-Arch...   
6  https://www.dice.com/jobs/detail/Network-Engin...   
7  https://www.dice.com/jobs/detail/Sr.-Web-Appli...   
8  https://www.dice.com/jobs/detail/Front-End-Dev...   
9  https://www.dice.com/jobs/detail/Application-S...   

                             company  \
0  Digital Intelligence Systems, LLC   
1  University of Chicago/IT Services   
2               Galaxy Systems, Inc.   
3                      TransTech LLC   
4                   Matrix Resources   
5                  Yash Technologies   
6                             Noble1   
7            Bluebeam Software, Inc.   
8                      

In [43]:
jobdomain=pd.read_csv(input_path+"frameworks_profile_job.csv",index_col='uniq_id')
print(jobdomain.index)
jobdomain.loc[jobdomain.index[1],:]

Index(['418ff92580b270ef4e7c14f0ddfc36b4', '8aec88cba08d53da65ab99cf20f6f9d9',
       '46baa1f69ac07779274bcd90b85d9a72', '3941b2f206ae0f900c4fba4ac0b18719',
       '45efa1f6bc65acc32bbbb953a1ed13b7', 'e0ac9d926dda5e95162ef05adea7318c',
       'e7e326053c586bd94e59f1fd74de4a1b', 'b0dadecf4c3c2beecb9c773ca11ecda4',
       '28f5e0c1cc3314813e674f0c32b04d1b', '95c9127e2770172f454f3b83981eaa88',
       ...
       '58920bf56320a30686832e5808bc10ba', 'a08db8b527bc82e3b464646732b837f1',
       'df62b009ce13775a21760bfdbdb095f1', '655351c499dcaf7f71efbf32bc2a76ee',
       'ee9f93d310f009dc9c0487ad2052b5ca', 'f9894a65fba83e7d248ba021253856b1',
       '9c5aa9bc9466d7269523f2b0f88d33f5', '120809874823e327aba99e9bd99e4861',
       'd3f81af99910f324ad114ccb4a82d71d', '28401827992cda8efc82d247cdc7fbfe'],
      dtype='object', name='uniq_id', length=15399)


agile                 NaN
angular               NaN
angularjs             NaN
asp.net mvc           NaN
aurelia               NaN
bottle                NaN
cakephp               NaN
cassandra             NaN
catalyst              NaN
cloudera              NaN
codeigniter           NaN
cordova               NaN
couchdb               NaN
django                NaN
dojo                  NaN
dropwizard            NaN
durandal              NaN
elm                   NaN
ember.js              NaN
express               NaN
flask                 NaN
flatiron              NaN
flex                  NaN
flink                 NaN
google web toolkit    NaN
grails                NaN
hadoop                NaN
halcyon               NaN
hive                  NaN
hpcc                  NaN
                     ... 
pyramid               NaN
rapidminer            NaN
react                 NaN
revel                 NaN
riot.js               NaN
ruby on rails         NaN
rum                   NaN
simplex     

Part A: Building the content analyzer

In [ ]:
#Identifying various categories in job postings

#Method1: Applying TF-IDF on the dataset
count=0
docs=[]
for i in range(len(job_description[:100])):
    #print(job_description[i])
    if(job_description[i]==np.nan):
        continue
    doc=[x for x in job_description[i].split(" ") if x not in stopwords_list]
    docs.append(" ".join(doc))
print(len(docs))
vectorizer=TfidfVectorizer(ngram_range=(1,2),max_df=0.6,max_features=50)
response=vectorizer.fit_transform(docs)
name_to_index=vectorizer.get_feature_names()
response=response.toarray()
scores=pd.DataFrame(data=response[:,:],index=range(len(response)),columns=name_to_index)
print(scores)
max_col_scores={}
#print(scores.iloc[0,:])
for col in range(len(scores.iloc[0,:])):
    col_score=sum(scores.iloc[:,col])
    max_col_scores[name_to_index[col]]=col_score
max_col_scores=sorted(max_col_scores.items(),reverse=True,key=lambda x:x[1])[:50]
print(max_col_scores)




In [ ]:
#Method 2: Use a separate domain field in the dataset.
#Use TF-DF on job titles
def cluster_job_titles():
    job_titles=df2.loc[:,'jobtitle'].tolist()
    #Tokenization   
    
    docs=[]
    for i in range(len(job_titles[:training_range])):
        #print(job_description[i])
        if(job_titles[i]==np.nan):
            continue
        doc=[x for x in job_description[i].split(" ") if x not in stopwords_list]
        docs.append(" ".join(doc))
    print(len(docs))
    vectorizer=TfidfVectorizer(ngram_range=(1,2),max_df=1.0,max_features=50)
    response=vectorizer.fit_transform(docs)
    model=KMeans(n_clusters=10,init='k-means++')
    model.fit(response)
    labels=model.labels_
    return labels
#name_to_index=vectorizer.get_feature_names()
#response=response.toarray()
#scores=pd.DataFrame(data=response[:,:],index=range(len(response)),columns=name_to_index)


In [ ]:
# labels=cluster_job_titles()
# cluster_mapping=dict()
# for i in range(len(labels)):
#     label=labels[i]
#     if(label not in cluster_mapping.keys()):
#         cluster_mapping[label]=[]
#     cluster_mapping[label].append(job_titles[i])
# print(cluster_mapping)
# print(len(cluster_mapping))

In [ ]:
# #Predefined categories
# #Compare similarities of word embeddings
# nlp=spacy.load('en_core_web_lg')
# # jobs=df2.loc[:,'jobtitle'].tolist()[:50]
# # jobs_descriptions=df2.loc[:,'jobdescription'].tolist()[:50]
# # for job in jobs:
# #     #print(job)
# #     doc=nlp(job)
# #     print(doc.ents)
#     #print("Entity is",ele.text,"Label",ele.label_)
# def check_threshold(threshold,ele):
#     if(ele[0]!=threshold[0][0] and abs(ele[1]-threshold[0][1])<0.03 and ele[1]>0.5):
#         return True
#     else:
#         return False
# def categorize(training_range):
#     job_id=df2.loc[:,'uniq_id'].tolist()[:training_range]
#     job_titles=df2.loc[:,'jobtitle'].tolist()[:training_range]
#     job_descriptions=df2.loc[:,'jobdescription'].tolist()[:training_range]
#     final_cat=pd.DataFrame(index=job_id)
#     #categories=['Network Engineer','Application Development','Big Data','Data Analyst','Software Developer','DevOps','Software Testing','Front End','Back End','Full Stack','Web Development','Information Security','Mobile developer','System Administrator','Business Analyst','Manager','Cloud']
#     categories=['Network Engineer','Full stack','QA/Test Developer','Enterprise application','DevOps','Mobile Developer','Back End','Database Administrator(DBA)','Front End','Game developer','System Administrator','Data Scientist','Business analyst','Sales professional','Product Manager','Information Security','Software Developer/Java Developer','Web Developer']
#     for category in categories:
#         final_cat[category]=np.nan
#     for job_t_d in list(zip(job_id,job_titles,job_descriptions)):
#         id_job=job_t_d[0]
#         job_i=job_t_d[1]
#         job_d=job_t_d[2]
#         job_title=nlp(job_i.lower())
#         job_description=nlp(job_d.lower())
#         match_cat_title=dict()
#         match_cat_description=dict()
#         for category in categories:
#             word=nlp(category.lower())
#             match_cat_title[category]=job_title.similarity(word)
#             match_cat_description[category]=job_description.similarity(word)
#         match_cat_title=sorted(match_cat_title.items(),key=lambda x:x[1],reverse=True)
#         match_cat_description=sorted(match_cat_description.items(),key=lambda x:x[1],reverse=True)
        
        
#         #a represents max
#         if(match_cat_title[0][1]>0.5 or match_cat_description[0][1]>0.5):
#             a=match_cat_title[0]
#             print(a)
#             match_cat_description=list(filter(lambda x: check_threshold(match_cat_title,x),match_cat_description))
#             if(len(match_cat_description)!=0):
#                 print(match_cat_description)
#                 print(id_job)
#                 #b=match_cat_description[0]
#                 final_cat.loc[id_job,a[0]]=1
#                 match_cat_description.extend([(match_cat_title[0][0],1)])
#                 sum_proportion=sum([x[1] for x in match_cat_description])
#                 for ele in match_cat_description:
#                     final_cat.loc[id_job,ele[0]]=ele[1]/sum_proportion
#             else:
#                 print(id_job)
#                 final_cat.loc[id_job,a[0]]=1

#         else:
#             print("not considering",job_i)
#         #print(match_cat)
#     return final_cat
# training_range=int(0.7*len(df2.loc[:,'uniq_id']))
# final_cat=categorize(training_range)

# print(final_cat)
# final_cat.to_csv("preprocessed_df.csv")

In [ ]:
#Extracting years of experience
nlp=spacy.load('en_core_web_lg')
job_description=df2.loc[:,'jobdescription'].tolist()
id_job=df2.loc[:,'uniq_id'].tolist()
experience_regex=['\d+ years \w+ $\.',r'\d+ experience']
matches=dict()
entities=dict()
for job_id,description in list(zip(id_job,job_description))[:10]:
    #l=re.findall(r"\w* experience[\S*\s*]\w*[.]",description)
    l=re.findall(r"[^.]*experience[^.]*\.",description)
    matches[job_id]=l    
    for string in matches[job_id]:
        print(string)
        doc=nlp(string)
        
        for token in doc:
            print(token.text,token.dep_,token.head.text)
print(matches)
#print(entities)

In [ ]:
#Identifying 

Part B: Profile Learner

In [ ]:
#Explicit/Implicit ways of identifying user preferences
#1) Use a like/dislike method to indicate user preference:
#Optimal method: Model based user-preference method to infer a score for each job(item) that the user has worked for in the past 

In [17]:
#LinkedIn Collab
files=glob.glob("../scraped profiles/*.json")
#print(files)
for file in files[:1]:
    f=open("../scraped profiles/"+file)
    data=json.load(f)
    print(data[0:2])

[{'positions': [{'title': 'Owner', 'is-current': True, 'company-name': 'AMBUS VENTURES'}], 'public-profile-url': '/pub/amark-a/39/565/222', 'location': 'Israel', 'first-name': 'Amark', 'num-connections': '13', 'last-name': 'A', 'industry': 'Wholesale'}, {'skills': ['Channel Management', 'Team Management', 'Key Account Management', 'People Management', 'Customer Relations'], 'positions': [{'title': 'Relationship Manager- WM', 'start-date': '2009-04-01', 'is-current': True, 'company-name': 'INGVysya Bank Ltd'}, {'title': 'Manager-Sales', 'end-date': '2009-03-01', 'start-date': '2008-05-01', 'company-name': 'Ikya Human Capital Solutions Pvt Ltd'}, {'title': 'Sales Manager', 'end-date': '2008-04-01', 'start-date': '2007-08-01', 'company-name': 'HDFC Bank ltd'}, {'title': 'Manager- Life insurance', 'end-date': '2007-08-01', 'start-date': '2003-08-01', 'company-name': 'ICICI Bank'}], 'public-profile-url': '/pub/amarnath-reddy-a/8/52a/280', 'location': 'Hyderabad Area, India', 'first-name': '